In [1]:
from typing import Dict, Set, Optional

from pyprojroot import here

In [3]:
def addRock(coordsList, cave: Dict[int, Set[int]]):
    currentX, currentY = coordsList[0]

    for nextX, nextY in coordsList[1:]:
        moveX = nextX - currentX
        moveY = nextY - currentY
        
        # moving along x-axis
        if moveX != 0:
            row = currentY
            start, stop = sorted([currentX, nextX])
            cols = range(start, stop + 1)

            for col in cols:
                cave.setdefault(col, set()).add(row)

        # moving along y-axis
        if moveY != 0:
            col = currentX
            start, stop = sorted([currentY, nextY])
            rows = range(start, stop + 1)
            cave.setdefault(col, set()).update(list(rows))

        currentX = nextX
        currentY = nextY


def buildCave(lines):
    cave = {}
    for line in lines:
        addRock(line, cave)

    return cave


def printCave(cave, sand):
    cols = sorted(cave.keys())
    allRows = [row for rows in cave.values() for row in rows] + [0]
    rows = list(range(min(allRows), max(allRows) + 1))

    for row in rows:
        line = ''

        for col in cols:

            # starting point
            if (col, row) == (500, 0):
                line += '+'

            # sand
            elif (col, row) in sand:
                line += 'o'

            # rock
            elif cave.get(col) and row in cave.get(col):
                line += "#"

            # air
            else:
                line += "\u00B7"

        print(line)

In [4]:
def dropSand(cave: dict, sand: set, floorY: int, col=500, row=0):

    # sand on floor
    if row == floorY - 1:
        cave.setdefault(col, set()).add(row)
        sand.add((col, row))
        return False

    # if space below
    if cave.get(col) and not row + 1 in cave[col] or not cave.get(col):
        return dropSand(cave, sand, floorY, col, row + 1)

    # if space diagonal left
    if cave.get(col - 1) and not row + 1 in cave[col - 1] or not cave.get(col - 1):
        return dropSand(cave, sand,floorY, col - 1, row + 1)

    # if space diagonal right
    if cave.get(col + 1) and not row + 1 in cave[col + 1] or not cave.get(col + 1):
        return dropSand(cave, sand, floorY, col + 1, row + 1)

    # sand at rest somewhere else
    cave[col].add(row)
    sand.add((col, row))

    # source blocked
    if (col, row) == (500, 0):
        return True

    # source not blocked
    return False


def flowSand(cave, floorY):
    sand = set()
    sandCount = 0
    sourceBlocked = False

    while not sourceBlocked:
        sourceBlocked = dropSand(cave, sand, floorY)
        sandCount += 1

    printCave(cave, sand)
    return sandCount

In [5]:
path = here('./14/input.txt')

with open(path, 'r') as fp:
    rawLines = [line.strip() for line in fp.readlines()]
    lines = [[[int(char) for char in coords.split(',')] for coords in line.split(' -> ')] for line in rawLines]
    
cave = buildCave(lines)
floorY = max([row for rows in cave.values() for row in rows]) + 2
flowSand(cave, floorY)

···················································································································································································+···················································································································································································
··················································································································································································ooo··················································································································································································
·················································································································································································ooooo··································································································

30762